In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file_paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        full_path = os.path.join(dirname, filename)
        file_paths.append(full_path)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


print(*file_paths[0:7], sep='\n')

/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
/kaggle/input/child-mind-institute-problematic-internet-use/train.csv
/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=00115b9f/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=001f3379/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0745c390/part-0.parquet


In [45]:
# Let's work with one parquet file and observe what it contains

first_df = pd.read_parquet(file_paths[4], engine='auto')
print(first_df.isna().sum())
first_df

step                   0
X                      0
Y                      0
Z                      0
enmo                   0
anglez                 0
non-wear_flag          0
light                  0
battery_voltage        0
time_of_day            0
weekday                0
quarter                0
relative_date_PCIAT    0
dtype: int64


,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,0.021536,0.022214,-1.022370,0.022853,-88.280762,0.0,53.000000,4188.000000,56940000000000,4,3,41.0
1,1,0.022005,0.022187,-1.019740,0.020231,-88.241707,0.0,51.666668,4188.166504,56945000000000,4,3,41.0
2,2,0.022240,0.022005,-1.019401,0.019893,-88.170067,0.0,50.333332,4188.333496,56950000000000,4,3,41.0
3,3,0.021589,0.022578,-1.018177,0.018667,-88.250031,0.0,50.500000,4188.500000,56955000000000,4,3,41.0
4,4,0.022005,0.023763,-1.014323,0.016848,-88.130775,0.0,33.166668,4181.000000,57235000000000,4,3,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43325,43325,-0.008333,-0.023620,-1.006901,0.007224,-88.595741,0.0,0.000000,3824.000000,57580000000000,6,3,85.0
43326,43326,0.048730,-0.076725,-0.953776,0.056839,-85.352219,0.0,0.000000,3824.000000,57640000000000,6,3,85.0
43327,43327,0.387370,0.793151,-0.402214,0.069961,-24.097908,0.0,0.000000,3824.000000,57645000000000,6,3,85.0
43328,43328,0.801953,0.501589,-0.040937,0.045489,-2.113776,0.0,0.000000,3824.000000,57650000000000,6,3,85.0


In [68]:
# Now let's start doing some data pre-processing
### 1. get rid of "probably" irrelevant features
first_df.drop(labels=['battery_voltage',"X","Y","Z"], axis=1)
# print(first_df.shape)
# first_df.columns

,step,enmo,anglez,non-wear_flag,light,time_of_day,weekday,quarter,relative_date_PCIAT
0,0,0.022853,-88.280762,0.0,53.000000,56940000000000,4,3,41.0
1,1,0.020231,-88.241707,0.0,51.666668,56945000000000,4,3,41.0
2,2,0.019893,-88.170067,0.0,50.333332,56950000000000,4,3,41.0
3,3,0.018667,-88.250031,0.0,50.500000,56955000000000,4,3,41.0
4,4,0.016848,-88.130775,0.0,33.166668,57235000000000,4,3,41.0
...,...,...,...,...,...,...,...,...,...
43325,43325,0.007224,-88.595741,0.0,0.000000,57580000000000,6,3,85.0
43326,43326,0.056839,-85.352219,0.0,0.000000,57640000000000,6,3,85.0
43327,43327,0.069961,-24.097908,0.0,0.000000,57645000000000,6,3,85.0
43328,43328,0.045489,-2.113776,0.0,0.000000,57650000000000,6,3,85.0


In [67]:
help(pd.DataFrame.drop)

Help on function drop in module pandas.core.frame:

drop(self, labels: 'IndexLabel | None' = None, *, axis: 'Axis' = 0, index: 'IndexLabel | None' = None, columns: 'IndexLabel | None' = None, level: 'Level | None' = None, inplace: 'bool' = False, errors: 'IgnoreRaise' = 'raise') -> 'DataFrame | None'
    Drop specified labels from rows or columns.
    
    Remove rows or columns by specifying label names and corresponding
    axis, or by directly specifying index or column names. When using a
    multi-index, labels on different levels can be removed by specifying
    the level. See the :ref:`user guide <advanced.shown_levels>`
    for more information about the now unused levels.
    
    Parameters
    ----------
    labels : single label or list-like
        Index or column labels to drop. A tuple will be used as a single
        label and not treated as a list-like.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Whether to drop labels from the index (0 or 'index') or


In [77]:
#1. Check missing values in all parquet folders:
#2. Normalize the parquet data
#3. Any more cleaning?
df = pd.DataFrame()
i=0
for file_path in file_paths[4:]:
    print(df)
    print(pd.read_parquet(file_path, engine='auto'))
    pd.concat([df,pd.read_parquet(file_path, engine='auto')])
    
    if i == 10:
        break
    i+=1

Empty DataFrame
Columns: []
Index: []
        step         X         Y         Z      enmo     anglez  \
0          0  0.021536  0.022214 -1.022370  0.022853 -88.280762   
1          1  0.022005  0.022187 -1.019740  0.020231 -88.241707   
2          2  0.022240  0.022005 -1.019401  0.019893 -88.170067   
3          3  0.021589  0.022578 -1.018177  0.018667 -88.250031   
4          4  0.022005  0.023763 -1.014323  0.016848 -88.130775   
...      ...       ...       ...       ...       ...        ...   
43325  43325 -0.008333 -0.023620 -1.006901  0.007224 -88.595741   
43326  43326  0.048730 -0.076725 -0.953776  0.056839 -85.352219   
43327  43327  0.387370  0.793151 -0.402214  0.069961 -24.097908   
43328  43328  0.801953  0.501589 -0.040937  0.045489  -2.113776   
43329  43329  0.792144  0.170486 -0.413108  0.063103 -16.771301   

       non-wear_flag      light  battery_voltage     time_of_day  weekday  \
0                0.0  53.000000      4188.000000  56940000000000        4   
1  